# SPR 2026 - Ensemble

Combina predições de múltiplos modelos.

**Estratégias:**
- Votação majoritária
- Média ponderada de probabilidades

**Formato:** Code Competition (Kaggle) / Google Colab

In [ ]:
# ============================================================
# SETUP - Ambiente e Dados
# ============================================================
import os
import sys

# Verificar Colab PRIMEIRO (mais confiável)
IS_COLAB = 'google.colab' in sys.modules
IS_KAGGLE = os.path.exists('/kaggle/input') and not IS_COLAB

print(f"Ambiente: {'Kaggle' if IS_KAGGLE else 'Colab' if IS_COLAB else 'Local'}")

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    DRIVE_BASE = '/content/drive/MyDrive/SPR_2026_outputs'
    DATA_DIR = f'{DRIVE_BASE}/data'
    OUTPUT_DIR = DRIVE_BASE
    
    # Verificar se dados existem no Drive
    if not os.path.exists(f'{DATA_DIR}/train.csv'):
        print("⚠️ Dados não encontrados no Drive!")
        print("Execute primeiro o notebook 00_download_data.ipynb")
        raise FileNotFoundError(f"Arquivo não encontrado: {DATA_DIR}/train.csv")
elif IS_KAGGLE:
    DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
    OUTPUT_DIR = '/kaggle/working'
else:
    DATA_DIR = '../data'
    OUTPUT_DIR = '../submissions'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import mode
import warnings
warnings.filterwarnings('ignore')

## 1. Carregar Submissões dos Modelos

Coloque os arquivos de submissão de cada modelo no mesmo diretório ou ajuste os paths.

In [ ]:
# Lista de submissões (arquivos salvos no Google Drive pelos outros notebooks)
submission_files = [
    'submission_tfidf.csv',
    'submission_word2vec.csv',
    'submission_bertimbau.csv',
    'submission_flan_t5.csv',
    'submission_distilbert.csv',
    'submission_deberta.csv',
    'submission_sbert.csv',
]

# Pesos para cada modelo (ajuste baseado em OOF F1)
weights = {
    'submission_tfidf.csv': 0.10,
    'submission_word2vec.csv': 0.10,
    'submission_bertimbau.csv': 0.20,
    'submission_flan_t5.csv': 0.15,
    'submission_distilbert.csv': 0.10,
    'submission_deberta.csv': 0.25,
    'submission_sbert.csv': 0.10,
}

# Carregar submissões disponíveis do OUTPUT_DIR (Google Drive)
submissions = {}
for f in submission_files:
    filepath = os.path.join(OUTPUT_DIR, f)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        submissions[f] = df
        print(f"Loaded: {f} ({len(df)} rows)")
    else:
        print(f"Not found: {f}")

print(f"\nTotal modelos carregados: {len(submissions)}")

## 2. Votação Majoritária

In [ ]:
if len(submissions) >= 2:
    # Alinhar por ID
    first_key = list(submissions.keys())[0]
    ids = submissions[first_key]['ID'].values
    
    # Criar matriz de predições
    preds_matrix = np.column_stack([
        submissions[k].set_index('ID').loc[ids, 'target'].values
        for k in submissions.keys()
    ])
    
    print(f"Matriz de predições: {preds_matrix.shape}")
    
    # Votação majoritária
    ensemble_preds = mode(preds_matrix, axis=1).mode.flatten()
    
    # Criar submissão
    ensemble_vote = pd.DataFrame({'ID': ids, 'target': ensemble_preds})
    ensemble_vote.to_csv(os.path.join(OUTPUT_DIR, 'submission_ensemble_vote.csv'), index=False)
    
    print("\nVotação Majoritária:")
    print(ensemble_vote['target'].value_counts().sort_index())
else:
    print("Necessário pelo menos 2 submissões")

## 3. Média Ponderada (se tiver probabilidades)

In [ ]:
# Se você salvou probabilidades ao invés de classes,
# pode fazer a média ponderada aqui.
# Exemplo com probabilidades:

# proba_files = [
#     ('model1_proba.npy', 0.4),
#     ('model2_proba.npy', 0.35),
#     ('model3_proba.npy', 0.25),
# ]
# 
# weighted_proba = np.zeros((n_samples, 7))
# for f, w in proba_files:
#     proba = np.load(f)
#     weighted_proba += w * proba
# 
# final_preds = np.argmax(weighted_proba, axis=1)

print("Para média ponderada, salve as probabilidades de cada modelo.")

## 4. Submissão Final

In [ ]:
# ============================================================
# Geração de Submissão Final
# ============================================================
# Use a submissão do ensemble ou a melhor individual
if len(submissions) >= 2:
    final_submission = ensemble_vote.copy()
    
    # SEMPRE salvar submission.csv no diretório atual (exigido pelo Kaggle)
    final_submission.to_csv('submission.csv', index=False)
    print("✅ submission.csv salvo no diretório atual")
    
    # Também salvar no OUTPUT_DIR para persistência (Colab/Local)
    if not IS_KAGGLE:
        submission_path = os.path.join(OUTPUT_DIR, 'submission_ensemble.csv')
        final_submission.to_csv(submission_path, index=False)
        print(f"✅ Cópia salva em: {submission_path}")
    
    print(f"\nDistribuição das predições (ensemble):")
    print(final_submission['target'].value_counts().sort_index())
else:
    print("Use a melhor submissão individual")

In [ ]:
# Download no Colab (opcional)
if IS_COLAB and os.path.exists('submission.csv'):
    from google.colab import files
    files.download('submission.csv')